<a href="https://colab.research.google.com/github/mileself/High-Quality-Momentum-hqm/blob/main/Quantitative_Momentum_Investing_v2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy import stats

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrety import IEX_CLOUD_API_TOKEN

In [3]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2756315101600,
 'week52high': 197.7,
 'week52low': 123.48,
 'week52highSplitAdjustOnly': 198.23,
 'week52lowSplitAdjustOnly': 124.17,
 'week52change': 0.1558078854550462,
 'sharesOutstanding': 15634232000,
 'float': 0,
 'avg10Volume': 75741659,
 'avg30Volume': 59199310,
 'day200MovingAvg': 174.34,
 'day50MovingAvg': 182.72,
 'employees': 164000,
 'ttmEPS': 5.95,
 'ttmDividendRate': 0.9368421399305272,
 'dividendYield': 0.005313908904881038,
 'nextDividendDate': '',
 'exDividendDate': '2023-08-11',
 'nextEarningsDate': '2023-11-02',
 'peRatio': 29.08732694807936,
 'beta': 1.3165272206357683,
 'maxChangePercent': 67.72758459379386,
 'year5ChangePercent': 2.3394262551261047,
 'year2ChangePercent': 0.20028076902390568,
 'year1ChangePercent': 0.08798550504620395,
 'ytdChangePercent': 0.36451306310350495,
 'month6ChangePercent': 0.192060323796851,
 'month3ChangePercent': -0.038160950643962965,
 'month1ChangePercent': -0.008380673828674134,
 'day30Ch

In [4]:
def chunks(lst,n):
  for i in range(0, len(lst), n):
    yield lst[i:i + n]

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
  symbol_strings.append(','.join(symbol_groups[i]))
  print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares To Buy']

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTL,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCA,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETFC,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD
GIS,GL,GLW,GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HFC,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILM

In [ ]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
  batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
  data = requests.get(batch_api_call_url).json()
  for symbol in symbol_string.split(','):
    if symbol == 'DISCA' or symbol == 'HFC' or symbol == 'VIAC' or symbol == 'WLTW':
           continue
    hqm_dataframe = hqm_dataframe.append(
        pd.Series(
            [
                symbol,
                data[symbol]['price'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
                'N/A'
            ],
            index=hqm_columns
        ),
        ignore_index = True
    )
hqm_dataframe

In [29]:
time_periods = [
    'One-Year',
    'Six-Month',
    'Three-Month',
    'One-Month'
]

for row in hqm_dataframe.index:
    for time_period in time_periods:
        if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0

for row in hqm_dataframe.index:
  for time_period in time_periods:
    hqm_dataframe.loc[row, f'{time_period} Return Percentile'] = stats.percentileofscore(hqm_dataframe[f'{time_period} Price Return'], hqm_dataframe.loc[row, f'{time_period} Price Return'])

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,113.430,N/A,-0.188215,14.97006,-0.166081,7.58483,-0.046775,26.546906,-0.110988,5.389222,N/A
1,AAL,13.305,N/A,-0.024879,38.922156,-0.087322,15.768463,-0.119226,8.582834,-0.098403,8.582834,N/A
2,AAP,57.930,N/A,-0.672006,0.598802,-0.544006,0.399202,-0.137931,6.586826,-0.205036,1.397206,N/A
3,AAPL,174.200,N/A,0.087986,60.678643,0.19206,77.44511,-0.038161,29.94012,-0.008381,59.88024,N/A
4,ABBV,151.160,N/A,0.091997,61.477046,0.017388,42.115768,0.095803,77.045908,-0.019122,48.702595,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,129.070,N/A,0.112489,65.06986,0.037569,47.105788,-0.044344,27.944112,-0.041282,31.936128,N/A
497,ZBH,120.630,N/A,0.025977,52.295409,-0.013486,30.938124,-0.102679,10.578842,-0.020173,48.103792,N/A
498,ZBRA,250.100,N/A,-0.194132,13.772455,-0.127971,10.778443,-0.102432,10.778443,-0.040582,32.53493,N/A
499,ZION,36.140,N/A,-0.338243,4.99002,-0.053795,21.756487,0.240759,95.808383,-0.020351,47.904192,N/A


In [33]:
from statistics import mean

for row in hqm_dataframe.index:
  momentum_percentiles = []
  for time_period in time_periods:
    momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
  hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,113.430,N/A,-0.188215,14.97006,-0.166081,7.58483,-0.046775,26.546906,-0.110988,5.389222,13.622754
1,AAL,13.305,N/A,-0.024879,38.922156,-0.087322,15.768463,-0.119226,8.582834,-0.098403,8.582834,17.964072
2,AAP,57.930,N/A,-0.672006,0.598802,-0.544006,0.399202,-0.137931,6.586826,-0.205036,1.397206,2.245509
3,AAPL,174.200,N/A,0.087986,60.678643,0.19206,77.44511,-0.038161,29.94012,-0.008381,59.88024,56.986028
4,ABBV,151.160,N/A,0.091997,61.477046,0.017388,42.115768,0.095803,77.045908,-0.019122,48.702595,57.335329
...,...,...,...,...,...,...,...,...,...,...,...,...
496,YUM,129.070,N/A,0.112489,65.06986,0.037569,47.105788,-0.044344,27.944112,-0.041282,31.936128,43.013972
497,ZBH,120.630,N/A,0.025977,52.295409,-0.013486,30.938124,-0.102679,10.578842,-0.020173,48.103792,35.479042
498,ZBRA,250.100,N/A,-0.194132,13.772455,-0.127971,10.778443,-0.102432,10.778443,-0.040582,32.53493,16.966068
499,ZION,36.140,N/A,-0.338243,4.99002,-0.053795,21.756487,0.240759,95.808383,-0.020351,47.904192,42.61477


In [35]:
hqm_dataframe.sort_values('HQM Score', ascending=False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True)
hqm_dataframe

,index,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,193,FTI,20.790,N/A,1.367001,99.600798,0.476531,97.804391,0.473493,99.600798,0.140741,99.401198,99.101796
1,281,LLY,598.880,N/A,0.92177,99.001996,0.911456,99.800399,0.348689,98.403194,0.136868,99.201597,99.101796
2,273,LB,79.920,N/A,2.281287,100.0,0.815831,99.600798,0.215421,94.610778,0.079271,96.806387,97.754491
3,343,NVDA,454.900,N/A,2.095309,99.800399,0.954206,100.0,0.136561,87.225549,0.098365,98.003992,96.257485
4,66,BKR,36.240,N/A,0.513199,96.606786,0.308416,92.814371,0.259527,96.806387,0.04566,91.217565,94.361277
5,240,INTC,38.730,N/A,0.282686,88.423154,0.449713,97.005988,0.183555,91.616766,0.113786,98.802395,93.962076
6,165,ETN,221.410,N/A,0.573774,97.60479,0.350444,95.209581,0.207103,93.612774,0.043689,89.421158,93.962076
7,9,ADBE,553.655,N/A,0.367974,92.015968,0.646553,99.401198,0.142385,88.223553,0.066398,95.808383,93.862275
8,381,PSX,123.170,N/A,0.443014,93.812375,0.252973,86.027944,0.296164,97.005988,0.07547,96.606786,93.363273
9,313,MPC,155.480,N/A,0.597036,98.203593,0.226681,83.233533,0.393796,99.401198,0.045882,91.417166,93.063872


In [36]:
def portfolio_input():
  global portfolio_size
  portfolio_size = input('Enter the size of your portfolio: ')

  try:
    float(portfolio_size)
  except ValueError:
    print('That is not a number! \nPlease try again: ')
    portfolio_size = input('Enter the size of your portfolio: ')

portfolio_input()
print(portfolio_size)

Enter the size of your portfolio: 1000000
1000000


In [38]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
  hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])

hqm_dataframe

,index,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,193,FTI,20.790,962,1.367001,99.600798,0.476531,97.804391,0.473493,99.600798,0.140741,99.401198,99.101796
1,281,LLY,598.880,33,0.92177,99.001996,0.911456,99.800399,0.348689,98.403194,0.136868,99.201597,99.101796
2,273,LB,79.920,250,2.281287,100.0,0.815831,99.600798,0.215421,94.610778,0.079271,96.806387,97.754491
3,343,NVDA,454.900,43,2.095309,99.800399,0.954206,100.0,0.136561,87.225549,0.098365,98.003992,96.257485
4,66,BKR,36.240,551,0.513199,96.606786,0.308416,92.814371,0.259527,96.806387,0.04566,91.217565,94.361277
5,240,INTC,38.730,516,0.282686,88.423154,0.449713,97.005988,0.183555,91.616766,0.113786,98.802395,93.962076
6,165,ETN,221.410,90,0.573774,97.60479,0.350444,95.209581,0.207103,93.612774,0.043689,89.421158,93.962076
7,9,ADBE,553.655,36,0.367974,92.015968,0.646553,99.401198,0.142385,88.223553,0.066398,95.808383,93.862275
8,381,PSX,123.170,162,0.443014,93.812375,0.252973,86.027944,0.296164,97.005988,0.07547,96.606786,93.363273
9,313,MPC,155.480,128,0.597036,98.203593,0.226681,83.233533,0.393796,99.401198,0.045882,91.417166,93.063872
